In [1]:
import findspark
findspark.init('/home/ubuntu/spark-2.3.2-bin-hadoop2.7')
import pyspark

from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('rec_example').getOrCreate()

In [2]:
from pyspark.ml.recommendation import ALS
from pyspark.ml.evaluation import RegressionEvaluator

In [3]:
data = spark.read.csv('../Python-and-Spark-for-Big-Data/Spark_for_Machine_Learning/Recommender_Systems/movielens_ratings.csv', 
                      inferSchema=True, header=True)

In [4]:
data.show()

+-------+------+------+
|movieId|rating|userId|
+-------+------+------+
|      2|   3.0|     0|
|      3|   1.0|     0|
|      5|   2.0|     0|
|      9|   4.0|     0|
|     11|   1.0|     0|
|     12|   2.0|     0|
|     15|   1.0|     0|
|     17|   1.0|     0|
|     19|   1.0|     0|
|     21|   1.0|     0|
|     23|   1.0|     0|
|     26|   3.0|     0|
|     27|   1.0|     0|
|     28|   1.0|     0|
|     29|   1.0|     0|
|     30|   1.0|     0|
|     31|   1.0|     0|
|     34|   1.0|     0|
|     37|   1.0|     0|
|     41|   2.0|     0|
+-------+------+------+
only showing top 20 rows



In [5]:
data.describe().show()

+-------+------------------+------------------+------------------+
|summary|           movieId|            rating|            userId|
+-------+------------------+------------------+------------------+
|  count|              1501|              1501|              1501|
|   mean| 49.40572951365756|1.7741505662891406|14.383744170552964|
| stddev|28.937034065088994| 1.187276166124803| 8.591040424293272|
|    min|                 0|               1.0|                 0|
|    max|                99|               5.0|                29|
+-------+------------------+------------------+------------------+



In [6]:
train, test = data.randomSplit([.8, .2])

In [7]:
als = ALS(maxIter=5, regParam=.01, userCol='userId', itemCol='movieId', ratingCol='rating')

In [8]:
model = als.fit(train)

In [9]:
preds = model.transform(test)

In [10]:
preds.show()

+-------+------+------+-----------+
|movieId|rating|userId| prediction|
+-------+------+------+-----------+
|     31|   1.0|    27| 0.23671454|
|     85|   1.0|    28|  1.5975775|
|     85|   2.0|    20|  1.8202742|
|     85|   5.0|     8|  4.5953383|
|     85|   1.0|    23| 0.23035109|
|     65|   1.0|    16|  3.6920562|
|     65|   1.0|    19|  2.7753582|
|     65|   1.0|     4|  -1.837265|
|     65|   5.0|    23|0.059839606|
|     65|   1.0|     2|-0.18553406|
|     53|   3.0|    13|   1.532145|
|     53|   1.0|    25|   2.355551|
|     53|   3.0|    14| 0.74910885|
|     78|   1.0|    27| 0.54675126|
|     78|   1.0|    17|-0.12907021|
|     78|   1.0|     8| 0.14893007|
|     78|   1.0|    11|  1.4596665|
|     78|   1.0|     2|   0.892896|
|     34|   1.0|    28|  1.5461762|
|     34|   3.0|     3| -1.3739866|
+-------+------+------+-----------+
only showing top 20 rows



In [11]:
evaluator = RegressionEvaluator(metricName='rmse', labelCol='rating', predictionCol='prediction')

In [12]:
rmse = evaluator.evaluate(preds)

In [13]:
print('RMSE:')
print(rmse)

RMSE:
1.882968716014837


In [14]:
single_user = test.filter(test['userId']==11).select(['movieId', 'userId'])

In [15]:
single_user.show()

+-------+------+
|movieId|userId|
+-------+------+
|     40|    11|
|     45|    11|
|     62|    11|
|     75|    11|
|     78|    11|
|     90|    11|
+-------+------+



In [16]:
recs = model.transform(single_user)

In [17]:
recs.orderBy('prediction', ascending=False).show()

+-------+------+----------+
|movieId|userId|prediction|
+-------+------+----------+
|     90|    11|  6.158666|
|     62|    11| 3.9938128|
|     75|    11| 2.5850513|
|     45|    11| 1.6091075|
|     78|    11| 1.4596665|
|     40|    11|0.21324223|
+-------+------+----------+

